In [95]:
import pytesseract
import cv2
from PIL import Image 
import re
import pandas as pd
import fitz
import os
import boto3
import io
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np

In [96]:
Ima = "Makro Invoices/Makro Invoices/New/61590_B38245866 DEFAULT B0134 TYPE_20220322_6_FacturaCompra_page1.png"

In [97]:
im = Image.open(Ima)

In [98]:
buffered = io.BytesIO()
im.save(buffered, format='PNG')
client = boto3.client('textract')
response = client.analyze_document(
    Document={'Bytes': buffered.getvalue()},
    FeatureTypes=['TABLES']
)

In [99]:
def map_blocks(blocks, block_type):
    return {
        block['Id']: block
        for block in blocks
        if block['BlockType'] == block_type
    }

blocks = response['Blocks']
tables = map_blocks(blocks, 'TABLE')
cells = map_blocks(blocks, 'CELL')
words = map_blocks(blocks, 'WORD')
selections = map_blocks(blocks, 'SELECTION_ELEMENT')

def get_children_ids(block):
    for rels in block.get('Relationships', []):
        if rels['Type'] == 'CHILD':
            yield from rels['Ids']
            
dataframes = []

for table in tables.values():

    # Determine all the cells that belong to this table
    table_cells = [cells[cell_id] for cell_id in get_children_ids(table)]

    # Determine the table's number of rows and columns
    n_rows = max(cell['RowIndex'] for cell in table_cells)
    n_cols = max(cell['ColumnIndex'] for cell in table_cells)
    content = [[None for _ in range(n_cols)] for _ in range(n_rows)]

    # Fill in each cell
    for cell in table_cells:
        cell_contents = [
            words[child_id]['Text']
            if child_id in words
            else selections[child_id]['SelectionStatus']
            for child_id in get_children_ids(cell)
        ]
        i = cell['RowIndex'] - 1
        j = cell['ColumnIndex'] - 1
        content[i][j] = ' '.join(cell_contents)

    # We assume that the first row corresponds to the column names
    dataframe = pd.DataFrame(content[1:], columns=content[0])
    dataframes.append(dataframe)

In [100]:
len(dataframes)

1

In [101]:
d = dataframes[0]

In [102]:
d.head(20)

,Código Descripción,,,Kilos,luds,Precio,IGIO,Oto
0,Albarán: D22/9.045 -,(22/03/22) -,575.31€,,,,,
1,02107358/GREF GREFUSACO PALOM.SR/AL,C12X135G,,"0,000 1.00",12.00,0.720,"3,00",8.64
2,Lote: 26735,Fecha Cad:,07/22,Unds: 12.00,,,,
3,02110294 GREF GREFUTUBO SR/AL,C12X105G,,"0,000 1.00",12.00,0.720,"3,00",8.64
4,Lote: 26731,Fecha Cad:,07/22,Unds: 12.00,,,,
5,02110296 GREF GREFUT. PALOM.BBQ,SR/AL C12X125G,,0.000 2.00,24.00,0.720,"3,00",17.28
6,Lote: 26217,Fecha Cad:,07/22,Unds: 24.00,,,,
7,02111950/AGREF GUBLINS X-TREME SR Lote: 26761,1.10€ C18UX110G Fecha Cad:,07/22,0.000 1.00 Unds: 18.00,18.00,0.814,3.00,14.65
8,02111951 /GREF GUBLINS SR C/PVP 1.10€,C18X120G,,"0,000 1.00",18.00,0.814,"3,00",14.65
9,Lote: 26747,Fecha Cad:,07/22,Unds: 18.00,,,,


In [103]:
merged_col = False
print(type(d))
for i in (d.columns):
    if i == '':
        print(i)
        merged_col = True

<class 'pandas.core.frame.DataFrame'>




In [104]:
if "Código Descripción" in d.columns:
    print('HI')

HI


In [105]:
def get_desc(desc):
    return desc.split(" ")[0].strip(" ")

In [106]:
if merged_col:
    print("hi")
    if "Código Descripción" in d.columns:
        d['Codigo'] = d["Código Descripción"].apply(lambda x: f"{get_desc(x)}")
        d['Codigo']= d['Codigo'].str.replace('/', '')
        d['Codigo']= d['Codigo'].str.replace(':', '')
        d['Codigo']= d['Codigo'].str.replace('[A-Z]', '')
        d['Codigo']= d['Codigo'].str.replace('[a-z]', '')
        d['DESCRIPCIÓN']= d['Código Descripción'].str.replace('[0-9]', '')
        d['DESCRIPCIÓN']= d['DESCRIPCIÓN']+' '+d.iloc[:,1]
        d =d.drop(d.columns[[0, 1]],axis = 1) 
        first_column = d.pop('Codigo')
        d.insert(0,'Codigo',first_column)
        second_column = d.pop('DESCRIPCIÓN')
        d.insert(1,'DESCRIPCIÓN',second_column)
    else:
        d['Codigo'] = d["Código"].apply(lambda x: f"{get_desc(x)}")
        d['Codigo']= d['Codigo'].str.replace('/', '')
        d['Codigo']= d['Codigo'].str.replace(':', '')
        d['Codigo']= d['Codigo'].str.replace('[A-Z]', '')
        d['Codigo']= d['Codigo'].str.replace('[a-z]', '')
        d['DESCRIPCIÓN']= d['Descripción'].str.replace('[0-9]', '')
        d['DESCRIPCIÓN']= d['DESCRIPCIÓN']+' '+d.iloc[:,1]
        d =d.drop(d.columns[[0, 1]],axis = 1) 
        first_column = d.pop('Codigo')
        d.insert(0,'Codigo',first_column)
        second_column = d.pop('DESCRIPCIÓN')
        d.insert(1,'DESCRIPCIÓN',second_column)
        
        
#    d['Codigo'] = d["Código"].apply(lambda x: f"{get_desc(x)}")
#    d['Codigo']= d['Codigo'].str.replace('/', '')
#    d['Codigo']= d['Codigo'].str.replace('[A-Z]', '')
#    d['Codigo']= d['Codigo'].str.replace('[a-z]', '')
#    d['DESCRIPCIÓN']= d['Código'].str.replace('/', '')
#    d['DESCRIPCIÓN']= d['DESCRIPCIÓN'].str.replace('[0-9]', '')
#    d['DESCRIPCIÓN']= d['DESCRIPCIÓN']+d.iloc[:,1]
#    d =d.drop(d.columns[[0, 1]],axis = 1) 
#    first_column = d.pop('Codigo')
#    d.insert(0,'Codigo',first_column)
#    second_column = d.pop('DESCRIPCIÓN')
#    d.insert(1,'DESCRIPCIÓN',second_column)

hi


F:\anaconda3\envs\python_3_7\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys

F:\anaconda3\envs\python_3_7\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  

F:\anaconda3\envs\python_3_7\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == '__main__':



In [107]:
if merged_col:
    this_data = d
    print('Yes')

Yes


In [108]:
str1 ='DESCRIPCIÓN'
str2 = "Descrip. artículo"
str3 = "Artículo/ Cod.cliente"
str4 = 'DESCRIPCIÓN:'
str5 = "CONCEPTO"
str6 = "UDS"
str7 = "Concepto"
str8 = "Descripción"
str9 = "DESCRIPCION"   
str10 = "Description articulo"
str11 = "Description"
str12 = "DESCRIPCIÓN ARTICULO"
str13 = "Unidades Descripción"
str14 = "PRODUCTO"
str15 = "Articulo"
str16 = 'Denominación /Referencia'
str17 = 'DESCRIPCIÓN'
str18 = 'REF.'
str19 = "Codigo"
str20 = "Lote"
str21 = "MM Num. articulo"
zero = False
one = False
two = False
three = False

if not merged_col:
    if "ARTÍCULO" in dataframes[0].columns or "CODI EAN/CÓDIGO EAN" in dataframes[0].columns or "MM Num. artículo" in dataframes[0].columns  or 'CÓDIGO' in dataframes[0].columns or 'MM Num.' in dataframes[0].columns or str3 in dataframes[0].columns or str4 in dataframes[0].columns or str5 in dataframes[0].columns or str6 in dataframes[0].columns or str7 in dataframes[0].columns or str8 in dataframes[0].columns or str9 in dataframes[0].columns or str10 in dataframes[0].columns or str11 in dataframes[0].columns or str12 in dataframes[0].columns or str13 in dataframes[0].columns or str14 in dataframes[0].columns or str15 in dataframes[0].columns or str16 in dataframes[0].columns or str17 in dataframes[0].columns or str18 in dataframes[0].columns or str19 in dataframes[0].columns or str20 in dataframes[0].columns or str21 in dataframes[0].columns :
            this_data = pd.DataFrame(dataframes[0])
            print('---------0')
            zero = True
    if len(dataframes)>1: 
        if "ARTÍCULO" in dataframes[1].columns or "CODI EAN/CÓDIGO EAN" in dataframes[1].columns or "MM Num. artículo" in dataframes[1].columns or 'CÓDIGO' in dataframes[1].columns or 'MM Num.' in dataframes[1].columns or str3 in dataframes[1].columns or str4 in dataframes[1].columns or str5 in dataframes[1].columns or str6 in dataframes[1].columns or str7 in dataframes[1].columns or str8 in dataframes[1].columns or str9 in dataframes[1].columns or str10 in dataframes[1].columns or str11 in dataframes[1].columns or str12 in dataframes[1].columns or str13 in dataframes[1].columns or str14 in dataframes[1].columns or str15 in dataframes[1].columns or str16 in dataframes[1].columns or str17 in dataframes[1].columns or str18 in dataframes[1].columns or str19 in dataframes[1].columns or str20 in dataframes[1].columns or str21 in dataframes[1].columns:        
                this_data = pd.DataFrame(dataframes[1])
                print('---------1')
                one = True
    if len(dataframes)>2:        
        if "ARTÍCULO" in dataframes[2].columns or "CODI EAN/CÓDIGO EAN" in dataframes[2].columns or "MM Num. artículo" in dataframes[2].columns or 'CÓDIGO' in dataframes[2].columns or 'MM Num.' in dataframes[2].columns or str3 in dataframes[2].columns or str4 in dataframes[2].columns or str5 in dataframes[0].columns or str6 in dataframes[2].columns or str7 in dataframes[2].columns or str8 in dataframes[2].columns or str9 in dataframes[2].columns or str10 in dataframes[2].columns or str11 in dataframes[2].columns or str12 in dataframes[2].columns or str13 in dataframes[2].columns or str14 in dataframes[2].columns or str15 in dataframes[2].columns or str16 in dataframes[2].columns or str16 in dataframes[2].columns or str18 in dataframes[2].columns or str19 in dataframes[2].columns or str20 in dataframes[2].columns or str21 in dataframes[2].columns:
                this_data = pd.DataFrame(dataframes[2])
                print('---------2')
                two = True
    if len(dataframes)>3:
        if "ARTÍCULO" in dataframes[3].columns or "CODI EAN/CÓDIGO EAN" in dataframes[3].columns or "MM Num. artículo" in dataframes[3].columns or 'CÓDIGO' in dataframes[3].columns or 'MM Num.' in dataframes[3].columns or str3 in dataframes[3].columns or str4 in dataframes[3].columns or str5 in dataframes[3].columns or str6 in dataframes[3].columns or str7 in dataframes[3].columns or str8 in dataframes[3].columns or str9 in dataframes[3].columns or str10 in dataframes[3].columns or str11 in dataframes[3].columns or str12 in dataframes[3].columns or str13 in dataframes[3].columns or str14 in dataframes[3].columns or str15 in dataframes[3].columns or str16 in dataframes[3].columns or str16 in dataframes[3].columns or str18 in dataframes[3].columns or str19 in dataframes[3].columns or str20 in dataframes[3].columns or  str21 in dataframes[3].columns:
            this_data = pd.DataFrame(dataframes[3])
            print('---------3')
            three = True
else:
    pass
    
this_data


,Codigo,DESCRIPCIÓN,Kilos,luds,Precio,IGIO,Oto
0,á,Albarán: D/. - (22/03/22) -,,,,,
1,02107358,/GREF GREFUSACO PALOM.SR/AL C12X135G,"0,000 1.00",12.00,0.720,"3,00",8.64
2,,Lote: Fecha Cad:,Unds: 12.00,,,,
3,02110294,GREF GREFUTUBO SR/AL C12X105G,"0,000 1.00",12.00,0.720,"3,00",8.64
4,,Lote: Fecha Cad:,Unds: 12.00,,,,
5,02110296,GREF GREFUT. PALOM.BBQ SR/AL C12X125G,0.000 2.00,24.00,0.720,"3,00",17.28
6,,Lote: Fecha Cad:,Unds: 24.00,,,,
7,02111950,/AGREF GUBLINS X-TREME SR Lote: 1.10€ C18UX11...,0.000 1.00 Unds: 18.00,18.00,0.814,3.00,14.65
8,02111951,/GREF GUBLINS SR C/PVP .€ C18X120G,"0,000 1.00",18.00,0.814,"3,00",14.65
9,,Lote: Fecha Cad:,Unds: 18.00,,,,


In [109]:
below_table = ''
if len(dataframes)>1:
    if zero:
        below_table = dataframes[1]
        print('---------')
if len(dataframes)>2:    
    if one:
        below_table = dataframes[2]
        print('---------')
        
if len(dataframes)>3:
    if two:
        below_table = dataframes[3]
        print('---------')
if len(dataframes)>3:
    if three:
        below_table = dataframes[4]
        print('---------')

In [110]:
below_table

''

In [111]:
if len(this_data)>4:
    this_data.columns = this_data.columns.str.replace('SELECTED  ', '')
    this_data.columns = this_data.columns.str.replace('SELECTED ', '')
    this_data.columns = this_data.columns.str.replace('SELECTED', '')
    this_data.columns = this_data.columns.str.replace('NOT_', '')
    this_data.columns = this_data.columns.str.replace('NOT_SELECTED  ', '')
    this_data.columns = this_data.columns.str.replace('NOT_SELECTED', '')
    big = this_data.columns[0]
    cont = this_data.columns[2]
    lot = this_data.columns[1]
    prec = this_data.columns[3]
    cant = this_data.columns[-5]
    imp = this_data.columns[-2]
    importe = this_data.columns[-3]
    third = this_data.columns[-4]
    this_data = this_data.loc[(this_data[big] != "Número de pedido") & (this_data[big] != "*** Número de pedido") & (this_data[big]!="*** Número de") & (this_data[big]!="*** Número")]
    this_data = this_data.loc[(this_data[big] != "Entregado a: TABERNA") & (this_data[big] !="Entregado a: TASCA") & (this_data[big]!="Entregado a: TABERNA OSUNA")
          & (this_data[big]!= "Entregado a: TABERNA osuna")& (this_data[big]!="Fin de número") ]
    this_data = this_data.loc[(this_data[imp]!= "Spain Fecha:") & (this_data[imp]!= "Spain Fecha:0") & (this_data[imp]!= "Spain Fecha:1")]
    this_data = this_data.loc[this_data[cont] !="LOT:"]
    this_data = this_data.loc[this_data[cant] !="Total"]
    this_data = this_data.loc[this_data[cant]!="LA LAGUNA" ]
    this_data = this_data.replace('',np.nan)
    this_data = this_data.dropna(axis=0, thresh=3, how="any")
    limitPer = len(this_data) * .100
    this_data = this_data.dropna(thresh=limitPer, axis=1)
    this_data = this_data.replace(np.nan,'0')

In [112]:
big = this_data

In [113]:
this_data.head()

,Codigo,DESCRIPCIÓN,Kilos,luds,Precio,IGIO,Oto
1,02107358,/GREF GREFUSACO PALOM.SR/AL C12X135G,"0,000 1.00",12.00,0.720,"3,00",8.64
3,02110294,GREF GREFUTUBO SR/AL C12X105G,"0,000 1.00",12.00,0.720,"3,00",8.64
5,02110296,GREF GREFUT. PALOM.BBQ SR/AL C12X125G,0.000 2.00,24.00,0.720,"3,00",17.28
7,02111950,/AGREF GUBLINS X-TREME SR Lote: 1.10€ C18UX11...,0.000 1.00 Unds: 18.00,18.00,0.814,3.00,14.65
8,02111951,/GREF GUBLINS SR C/PVP .€ C18X120G,"0,000 1.00",18.00,0.814,"3,00",14.65


In [114]:
# for  second column
this_data["second_count"] = this_data.iloc[:, 1].str.count("\D")
this_data["second_length"] = this_data.iloc[:, 1].str.len()
this_data["second_digit_percentage"] = this_data["second_count"] / this_data["second_length"] * 100



In [115]:
# for third column
this_data["third_count"] = this_data.iloc[:, 2].str.count("\D")
this_data["third_length"] = this_data.iloc[:, 2].str.len()
this_data["third_digit_percentage"] = this_data["third_count"] / this_data["third_length"] * 100



In [116]:
len(this_data.columns)

13

In [117]:
this_data.head()

,Codigo,DESCRIPCIÓN,Kilos,luds,Precio,IGIO,Oto,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
1,02107358,/GREF GREFUSACO PALOM.SR/AL C12X135G,"0,000 1.00",12.00,0.720,"3,00",8.64,31,36,86.111111,3,10,30.0
3,02110294,GREF GREFUTUBO SR/AL C12X105G,"0,000 1.00",12.00,0.720,"3,00",8.64,25,30,83.333333,3,10,30.0
5,02110296,GREF GREFUT. PALOM.BBQ SR/AL C12X125G,0.000 2.00,24.00,0.720,"3,00",17.28,33,38,86.842105,3,10,30.0
7,02111950,/AGREF GUBLINS X-TREME SR Lote: 1.10€ C18UX11...,0.000 1.00 Unds: 18.00,18.00,0.814,3.00,14.65,51,59,86.440678,11,22,50.0
8,02111951,/GREF GUBLINS SR C/PVP .€ C18X120G,"0,000 1.00",18.00,0.814,"3,00",14.65,30,35,85.714286,3,10,30.0


In [118]:
big

,Codigo,DESCRIPCIÓN,Kilos,luds,Precio,IGIO,Oto,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
1,02107358,/GREF GREFUSACO PALOM.SR/AL C12X135G,"0,000 1.00",12.00,0.720,"3,00",8.64,31,36,86.111111,3,10,30.0
3,02110294,GREF GREFUTUBO SR/AL C12X105G,"0,000 1.00",12.00,0.720,"3,00",8.64,25,30,83.333333,3,10,30.0
5,02110296,GREF GREFUT. PALOM.BBQ SR/AL C12X125G,0.000 2.00,24.00,0.720,"3,00",17.28,33,38,86.842105,3,10,30.0
7,02111950,/AGREF GUBLINS X-TREME SR Lote: 1.10€ C18UX11...,0.000 1.00 Unds: 18.00,18.00,0.814,3.00,14.65,51,59,86.440678,11,22,50.0
8,02111951,/GREF GUBLINS SR C/PVP .€ C18X120G,"0,000 1.00",18.00,0.814,"3,00",14.65,30,35,85.714286,3,10,30.0
10,02111952,/GREF GUBLINS XL QUESO SR C/PVP 1.10€ C18X110G,"0,000 1.00",18.00,0.814,"3,00",14.65,38,46,82.608696,3,10,30.0
12,02111953,/ GREF BOOM TIJUANA C/PVP SR C16X85G,"0,000 1.00",16.00,0.740,"3,00",11.84,32,36,88.888889,3,10,30.0
14,02111953,/GREF BOOM TIJUANA C/PVP SR C16X85G,"0,000 0.44",7.00,0.000,"3,00",0.00,31,35,88.571429,3,10,30.0
16,02111954,/AGREF TIRAS BACON C/PVP SR C18X120G,"0,000 1.00",18.00,0.740,"3,00",13.32,32,37,86.486486,3,10,30.0
18,02111957,GREF STICKS KETC KETCHUP C/PVP SR C14X120G,"0,000 1.00",14.00,0.740,"3,00",10.36,38,43,88.372093,3,10,30.0


In [119]:
dftwo = False
# for  forth column
if len(big.columns)==3:
    this_data["forth_count"] = this_data.iloc[:, 3].str.count("\D")
    this_data["forth_length"] = this_data.iloc[:, 3].str.len()
    this_data["forth_digit_percentage"] = this_data["forth_count"] / this_data["forth_length"] * 100
    dftwo = True



In [120]:
df = this_data.query("second_digit_percentage>30.0")
df1 = this_data.query("third_digit_percentage>30.0")
if dftwo:
    df2 = this_data.query("forth_digit_percentage>60.0")


In [121]:
this_data

,Codigo,DESCRIPCIÓN,Kilos,luds,Precio,IGIO,Oto,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
1,02107358,/GREF GREFUSACO PALOM.SR/AL C12X135G,"0,000 1.00",12.00,0.720,"3,00",8.64,31,36,86.111111,3,10,30.0
3,02110294,GREF GREFUTUBO SR/AL C12X105G,"0,000 1.00",12.00,0.720,"3,00",8.64,25,30,83.333333,3,10,30.0
5,02110296,GREF GREFUT. PALOM.BBQ SR/AL C12X125G,0.000 2.00,24.00,0.720,"3,00",17.28,33,38,86.842105,3,10,30.0
7,02111950,/AGREF GUBLINS X-TREME SR Lote: 1.10€ C18UX11...,0.000 1.00 Unds: 18.00,18.00,0.814,3.00,14.65,51,59,86.440678,11,22,50.0
8,02111951,/GREF GUBLINS SR C/PVP .€ C18X120G,"0,000 1.00",18.00,0.814,"3,00",14.65,30,35,85.714286,3,10,30.0
10,02111952,/GREF GUBLINS XL QUESO SR C/PVP 1.10€ C18X110G,"0,000 1.00",18.00,0.814,"3,00",14.65,38,46,82.608696,3,10,30.0
12,02111953,/ GREF BOOM TIJUANA C/PVP SR C16X85G,"0,000 1.00",16.00,0.740,"3,00",11.84,32,36,88.888889,3,10,30.0
14,02111953,/GREF BOOM TIJUANA C/PVP SR C16X85G,"0,000 0.44",7.00,0.000,"3,00",0.00,31,35,88.571429,3,10,30.0
16,02111954,/AGREF TIRAS BACON C/PVP SR C18X120G,"0,000 1.00",18.00,0.740,"3,00",13.32,32,37,86.486486,3,10,30.0
18,02111957,GREF STICKS KETC KETCHUP C/PVP SR C14X120G,"0,000 1.00",14.00,0.740,"3,00",10.36,38,43,88.372093,3,10,30.0


In [122]:
this_data.shape

(17, 13)

In [123]:
desc = ''
if len(df) == len(this_data):
    desc = df.iloc[:,1]
    print(' I got in ')
elif len(df1)== len(this_data):
    desc = df1.iloc[:,2]
    print('I got it ')
elif len(df2)== len(this_data):
    desc = df2.iloc[:,3]
    print('I got it2 ')

 I got in 


In [124]:
desc

1                  /GREF GREFUSACO PALOM.SR/AL C12X135G
3                         GREF GREFUTUBO SR/AL C12X105G
5                 GREF GREFUT. PALOM.BBQ SR/AL C12X125G
7     /AGREF GUBLINS X-TREME SR Lote:  1.10€ C18UX11...
8                    /GREF GUBLINS SR C/PVP .€ C18X120G
10       /GREF GUBLINS XL QUESO SR C/PVP 1.10€ C18X110G
12                 / GREF BOOM TIJUANA C/PVP SR C16X85G
14                  /GREF BOOM TIJUANA C/PVP SR C16X85G
16                 /AGREF TIRAS BACON C/PVP SR C18X120G
18           GREF STICKS KETC KETCHUP C/PVP SR C14X120G
20                             GREF CONOS SR C/PVP CXG 
22                  GREF COMBINADO RED C/PVP SR C15X85G
24                      GREF PATATA OND.JAMON € SR C18U
26                      /GREF NACHOS TEX MEX SR C12X95G
28                    , IGREF CORTEZAS TRIGO SR C10X60G
30                 , GREF PATATA ARTESANA € SR C20X110G
32                      IGREF PAT.CAMPESINA SR C20X110G
Name: DESCRIPCIÓN, dtype: object

In [125]:
if len(desc)!=0:
    desc_data = pd.DataFrame(desc)
    desc_data.head()

In [126]:
code  = this_data.iloc[:, 0]
#codingo = this_data[code]
codingo_data  =  pd.DataFrame(code)
codingo_data


,Codigo
1,02107358
3,02110294
5,02110296
7,02111950
8,02111951
10,02111952
12,02111953
14,02111953
16,02111954
18,02111957


In [127]:
prec = []
str1 = "Precio"
str2 = "Base I"
str3 = "PVP"
str4 = 'Importe'
str5 = 'Euro Uni.'
str6 = "IMP"
price =''
for i in this_data.columns:
    ratio1 = fuzz.partial_ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        prec.append(i)
        price = i
        print("Im condition 1")
    ratio2 = fuzz.partial_ratio(str2.lower(),str(i).lower())
    if ratio2>=75:
        print(i)
        prec.append(i)
        price =i
        print("Im condition 2")
    ratio3 = fuzz.partial_ratio(str3.lower(),str(i).lower())
    if ratio3>=75:
        print("Im condition 3")
        print(i)
        prec.append(i)
        price =i
    ratio4 = fuzz.partial_ratio(str4.lower(),str(i).lower())
    if ratio4>=75:
        print("Im condition 4")
        print(i)
        prec.append(i)
        price =i
    ratio5 = fuzz.partial_ratio(str5.lower(),str(i).lower())
    if ratio5>=75:
        print("Im condition 5")
        print(i)
        prec.append(i)
        price =i
    ratio6 = fuzz.partial_ratio(str6.lower(),str(i).lower())
    if ratio6>=75:
        print("Im condition 6")
        print(i)
        prec.append(i)
        price =i
if len(prec)>1:
    price = prec[0]
else:
    pass

Precio
Im condition 1


In [128]:
if price !='':
    price_col = this_data[price]
    price_data = pd.DataFrame(price_col,columns=[price])
    price_data
else:
    print("Price is Empty")

In [129]:
str1 = "DTO"
str2 = "Descuento"
str3 = "DCTO"
str4 = "Desc"
disc = ''
for i in this_data.columns:
    ratio1 = fuzz.ratio(str1.lower(),str(i).lower())
    ratio2 = fuzz.ratio(str2.lower(),str(i).lower())
    ratio3 = fuzz.ratio(str3.lower(),str(i).lower())
    ratio4 = fuzz.ratio(str4.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        disc = i
        print("Im condition 1")
    
    elif ratio2>=75:
        print(i)
        disc = i
        print("Im condition 2")
    
    elif ratio3>=75:
        print(i)
        disc = i
        print("Im condition 3")
    
    elif ratio4>=75:
        print(i)
        disc = i
        print("Im condition 4")
    
print(disc)

In [130]:
if disc!='':
    disc_col = this_data[disc]
    disc_data = pd.DataFrame(disc_col,columns=[disc])
    disc_data

In [131]:
str1 = "Cantidad"
str2 = "Cant"
str3 = "Unid"
str4 = "Unidades"
str5 = "Uds"
str6 = "Peso"
cat = ''
for i in this_data.columns:
    ratio1 = fuzz.ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        cat = i
        print("Im condition 1")
    ratio2 = fuzz.ratio(str2.lower(),str(i).lower())
    if ratio2>=75:
        print(i)
        cat =i
        print("Im condition 2")
    ratio3 = fuzz.ratio(str3.lower(),str(i).lower())
    if ratio3>=75:
        print("Im condition 3")
        print(i)
        cat =i
    ratio4 = fuzz.ratio(str4.lower(),str(i).lower())
    if ratio4>=75:
        print("Im condition 4")
        print(i)
        cat =i
    ratio5 = fuzz.ratio(str5.lower(),str(i).lower())
    if ratio5>=75:
        print("Im condition 5")
        print(i)
        cat =i
    ratio6 = fuzz.ratio(str6.lower(),str(i).lower())
    if ratio6>=75:
        print("Im condition 6")
        print(i)
        cat =i

Im condition 5
luds


In [132]:
if cat!='':
    Cantidad = this_data[cat]
    Cantidad_data = pd.DataFrame(Cantidad,columns=[cat])
    Cantidad_data
else:
    print("quantity is empty")

In [133]:
str1 = 'IMPUESTO'
imposto = ''
for i in this_data.columns:
    ratio1 = fuzz.partial_ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        imposto = i
print(imposto)

In [134]:
if imposto!='':
    imposto_col = this_data[imposto]
    imposto_data = pd.DataFrame(imposto_col,columns=[imposto])
    imposto_data

In [135]:
str1 = 'Tasa'
str2 = 'TIPO'
tasa = ''
for i in this_data.columns:
    ratio1 = fuzz.ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        tasa = i
    ratio2 = fuzz.ratio(str2.lower(),str(i).lower())
    if ratio2>=75:
        print(i)
        tasa = i
print(tasa)

In [136]:
if tasa!='':
    tasa_col = this_data[tasa]
    tasa_data = pd.DataFrame(tasa_col,columns=[tasa])
    tasa_data

In [141]:
if disc =='' and imposto =='' and tasa =='' and cat=='' and price == '':
    df2 = pd.concat([codingo_data,desc_data],axis=1, join='inner')
    df2['Price'] = 0
    df2['Quantity'] = 0
    df2['Discount'] = 0
    df2['IMPUESTO']=0
    df2['Tasa']=0
elif desc !='' and imposto =='' and tasa =='' and disc=='' and cat=='' and price == '':
    df2 = pd.concat([codingo_data,desc_data],axis=1, join='inner')
    #df2["Description"]= ''
    df2['Price'] = 0
    df2['Quantity'] = 0
    df2['Discount'] = 0
    df2['IMPUESTO']=0
    df2['Tasa']=0

elif desc!='' and code!='' and price!='' and cat!='' and disc=='' and imposto=='' and tasa =='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data])
    df2['Discount'] = 0
    df2['IMPUESTO']=0
    df2['Tasa']=0
    
elif disc!='' and imposto =='' and tasa =='':
    disc_col = this_data[disc]
    disc_data = pd.DataFrame(disc_col,columns=[disc])
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,disc_data],axis=1, join='inner')
    df2['IMPUESTO']=0
    df2['Tasa']=0
    
elif disc!='' and imposto!='' and tasa=='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,disc_data,imposto_data],axis=1, join='inner')
    df2['Tasa']=0
    
elif disc!='' and imposto=='' and tasa!='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,disc_data,tasa_data],axis=1, join='inner')
    df2['IMPUESTO']=0
    
elif disc=='' and imposto!='' and tasa!='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,imposto_data,tasa_data],axis=1, join='inner')
    df2['Discount']=0
    
elif disc=='' and imposto=='' and tasa!='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,tasa_data],axis=1, join='inner')
    df2['Discount']=0
    df2['IMPUESTO']=0
    
elif disc=='' and imposto!='' and tasa=='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,imposto_data],axis=1, join='inner')
    df2['Discount']=0
    df2['Tasa']=0

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
df2